In [1]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats

from helper import *

# Manual Analysis

We measured three behaviors ("Approach hands to food (n)," "Bring the food to the mouth (n)," and "Drop the food (n)") and "Food intake (g)" in the manual analysis. One can use the sample in `Examples/Palatable_Manual.csv` to reproduce the same results as below. First, read the csv file as follows:

In [2]:
df = pd.read_csv('Examples/Palatable_Manual.csv', index_col=0, header=[0, 1])
df

A            B            C        
                                 CNO Vehicle  CNO Vehicle  CNO Vehicle
Approach hands to food (n)        30      21   37      27   80      33
Bring the food to the mouth (n)  588     340  405     236  303     217
Drop the food (n)                  3      53   20      33   15      53
Food intake (g)                  214      76  162     140  622     266

Comparing CNO-injected and control subjects for each index by the paired t-test, we have:

In [3]:
print('{:>35}{:>10}'.format('Index', 'p-value'))

for idx in df.index:
    data = df.loc[idx].values.reshape(3, 2)
    p = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
    print('{:>35}{:>10.3f}'.format(idx, p))

                              Index   p-value
         Approach hands to food (n)     0.221
    Bring the food to the mouth (n)     0.070
                  Drop the food (n)     0.091
                    Food intake (g)     0.221


We also divided the values of CNO-injected subjects by the corresponding values of control subjects, and compared the ratios with 1, so that we can observe the rate of change caused by CNO injection.

In [4]:
print('{:>35}{:>10}'.format('Index', 'p-value'))

for idx in df.index:
    data = df.loc[idx].values.reshape(3, 2)
    p = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1], popmean=1).pvalue
    print('{:>35}{:>10.3f}'.format(idx, p))

                              Index   p-value
         Approach hands to food (n)     0.163
    Bring the food to the mouth (n)     0.030
                  Drop the food (n)     0.050
                    Food intake (g)     0.155


# Deep Learning-based analysis

We measured three behaviors ("Tray Approach (n)," "Bout (n)," and "Drop the food (n)") and "Duration in Food Zone (g)" in the deep learning-based analysis. One can use the samples in `Examples/Palatable_DLC_MonkeyX_Y.csv (X = A / B / C, Y = CNO / Vehicle)` to reproduce the same results as below (those are raw outputs from DeepLabCut). We computed some parameters for each video, such as `Examples/Palatable_DLC_params.csv`.

* `fz x` is the x-coordinate of the food zone.
* `trayx` and `trayy` are the x- and y-coordinate of the tray, respectively.
* `tray-hand dist` is the threshold distance between the tray and hand, which is a criterion for "Tray Approach."
* `tray-mouth dist` is the threshold distance between the tray and mouth, which is a criterion for "Bout."

In [5]:
params_df = pd.read_csv('Examples/Palatable_DLC_params.csv', index_col=[0, 1], header=0)
params_df

fz x  trayx  trayy  tray-hand dist  tray-mouth dist
A CNO       847   1137    459       97.693398        22.203603
  Vehicle   843   1142    455      106.404887        20.248457
B CNO       809   1133    450      120.904921        21.189620
  Vehicle   866   1147    454       94.540996        21.587033
C CNO       906   1168    457       94.366308        22.472205
  Vehicle   845   1139    455       89.157165        17.492856

Based on those parameters, we computed each index for each video. Refer to `helper.py` for the detailed description of each function.

In [6]:
cases = [(i, j) for i in ['A', 'B', 'C'] for j in ['CNO', 'Vehicle']]
filenames = ['Examples/Palatable_DLC_Monkey{}_{}.csv'.format(i, j) for (i, j) in cases]

values = np.array([])

for case, filepath in zip(cases, filenames):
    # Approach
    params = tuple(params_df.loc[case, ['trayx', 'trayy', 'tray-hand dist']])
    coords, _, _ = return_approach(filepath, params, ll_crit=0.9, absolute=True, interval=0.2, FPS=24.0)
    appr = coords['Approach'].sum()

    # Bout
    params = tuple(params_df.loc[case, ['trayx', 'trayy', 'tray-mouth dist', 'tray-hand dist']])
    coords, _, _ = return_bout1(filepath, params, latency1=3.0, latency2=1.0, ll_crit=0.9, interval=0.2, FPS=24.0)
    bout = coords['Bout'].sum()

    # In Food Zone
    fz_x = params_df.loc[case, 'fz x']
    coords, _, _ = return_infz(filepath, fz_x, ll_crit=0.9, absolute=True)
    infz = coords['In'].sum() / 24


    values = np.concatenate((values, np.array([appr, bout, infz])))

values = values.reshape(3, 2, 3)

values

array([[[ 55.        ,  31.        , 866.83333333],
        [ 50.        ,  20.        , 217.58333333]],

       [[121.        ,  74.        , 828.54166667],
        [ 80.        ,  44.        , 204.        ]],

       [[268.        , 112.        , 520.45833333],
        [ 66.        ,  18.        , 309.375     ]]])

As in the manual analysis, we conducted two ways of comparison: first, we compared CNO-injected and control subjects for each index by the paired t-test...

In [7]:
indices = ["Tray Approach (n)", "Bout (n)", "Duration in Food Zone (s)"]

print('{:>30}{:>10}'.format('Index', 'p-value'))

for i, idx in enumerate(indices):
    data = values[:, :, i]
    p = scipy.stats.ttest_rel(data[:, 0], data[:, 1]).pvalue
    print('{:>30}{:>10.3f}'.format(idx, p))

                         Index   p-value
             Tray Approach (n)     0.306
                      Bout (n)     0.215
     Duration in Food Zone (s)     0.073


and second, we observed the rate of change caused by CNO injection in each subject, for each index.

In [8]:
print('{:>30}{:>10}'.format('Index', 'p-value'))

for i, idx in enumerate(indices):
    data = values[:, :, i]
    p = scipy.stats.ttest_1samp(data[:, 0] / data[:, 1], popmean=1).pvalue
    print('{:>30}{:>10.3f}'.format(idx, p))

                         Index   p-value
             Tray Approach (n)     0.317
                      Bout (n)     0.296
     Duration in Food Zone (s)     0.103
